<a href="https://colab.research.google.com/github/tarupathak30/rag-astronomy-chatbot-/blob/main/bot_astro_exoplanets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, shutil

# create folder
folder_name = "exoplanet_data"
os.makedirs(folder_name, exist_ok=True)

In [ ]:
from google.colab import files
files.upload()

In [ ]:
json_path = os.path.join(folder_name, "planets.json")

In [ ]:
source = "/content/exoplanets.json"  # your existing file
destination = os.path.join(folder_name, "/content/exoplanet_data/exoplanets.json")

shutil.move(source, destination)

print("File moved:", destination)

In [ ]:
!pip install langchain_community faiss-cpu

In [ ]:
import os, json, re, math, torch, faiss
import numpy as np
from typing import List, Dict, Any

from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

In [ ]:
MODEL_ID       = "google/gemma-2-2b-it"
EMBED_MODEL    = "sentence-transformers/all-mpnet-base-v2"
DEVICE         = "cuda" if torch.cuda.is_available() else "cpu"
TOP_K          = 5
DEBUG          = True   # set True to inspect intermediate steps



In [ ]:
from huggingface_hub import login

login()   # It will ask you to paste your token


In [ ]:
print("Loading Gemma...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
llm = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16 if DEVICE=="cuda" else torch.float32,
    device_map="auto" if DEVICE=="cuda" else None
)
print("Loaded Gemma on", DEVICE)


In [ ]:
print("Loading embedding model...")
embed_model = SentenceTransformer(EMBED_MODEL, device=DEVICE)
embed_dim = embed_model.get_sentence_embedding_dimension()
print("Embedding dimension =", embed_dim)

In [ ]:

# ---------- UTILITIES ----------
def planet_to_text(obj):
    name = obj.get("planet_name", "Unnamed Planet")

    profile = obj.get("planet_profile", {})
    star = obj.get("host_star", {})
    disc = obj.get("discovery", {})
    env = obj.get("environment", {})

    parts = [
        f"The planet {name} has a radius of {profile.get('radius_earth_radii')} Earth radii "
        f"and a mass of {profile.get('mass_earth_masses')} Earth masses.",
        f"It orbits its star every {profile.get('orbital_period_days')} days "
        f"at a distance of {profile.get('semi_major_axis_au')} AU.",
        f"The host star {star.get('name')} is a {star.get('spectral_type')} type star "
        f"with a temperature of {star.get('temperature_k')} K.",
        f"This planet was discovered in {disc.get('year')} "
        f"using the {disc.get('method')} method at {disc.get('facility')}.",
        f"The equilibrium temperature of the planet is {env.get('equilibrium_temperature_k')} K "
        f"and it lies at a distance of {env.get('distance_pc')} parsecs from Earth."
    ]

    return " ".join(parts)



def chunk_text(text, max_len=400):
    words = text.split()
    chunks, cur = [], []

    for w in words:
        cur.append(w)
        if len(cur) >= max_len:
            chunks.append(" ".join(cur))
            cur = []

    if cur:
        chunks.append(" ".join(cur))

    return chunks


In [ ]:
def preview_chunks(data_list):
    for obj in data_list[:3]:  # preview first 3 planets
        text = planet_to_text(obj)
        chunks = chunk_text(text, max_len=500)

        print(f"\n======== {obj.get('planet_name')} ========\n")
        for i, chunk in enumerate(chunks):
            print(f"\n--- Chunk {i} ---\n{chunk}\n")



with open("/content/exoplanet_data/exoplanets.json", "r", encoding="utf-8") as f:
    data_list = json.load(f)

preview_chunks(data_list)

In [ ]:

# ---------- INDEX BUILDING ----------
def build_index(planets):
    embeddings = []
    meta_objects = []

    for planet in planets:
        text = planet_to_text(planet)
        chunks = chunk_text(text)

        for chunk in chunks:
            emb = embed_model.encode([chunk], convert_to_numpy=True)[0].astype("float32")
            embeddings.append(emb)
            meta_objects.append(planet)

    embeddings = np.vstack(embeddings)

    print("Total chunks:", len(embeddings))
    print("Embedding shape:", embeddings.shape)

    faiss.normalize_L2(embeddings)

    index = faiss.IndexFlatIP(embeddings.shape[1])
    index.add(embeddings)

    meta = {"objects": meta_objects}
    print("FAISS index built:", index.ntotal)
    return index, meta



In [ ]:

# ---------- RETRIEVAL ----------
def retrieve_objects(query, index, meta, top_k=5):
    q_emb = embed_model.encode([query], convert_to_numpy=True).astype("float32")
    faiss.normalize_L2(q_emb)

    D, I = index.search(q_emb, top_k)

    results = []
    seen = set()

    for idx in I[0]:
        planet = meta["objects"][idx]
        pid = id(planet)
        if pid not in seen:
            seen.add(pid)
            results.append(planet)

    return results


In [ ]:
import re

def extract_year(q):
    match = re.search(r"\b(19|20)\d{2}\b", q)
    return int(match.group()) if match else None


In [ ]:

# ---------- FACT CHECK LOGIC ----------
def fact_check(question, candidates):
    q = question.lower()

    # Longest orbital period
    if "longest orbital period" in q:
        return max(candidates, key=lambda c: c["planet_profile"].get("orbital_period_days", -1))

    # Shortest orbital period
    if "shortest orbital period" in q:
        return min(candidates, key=lambda c: c["planet_profile"].get("orbital_period_days", 1e18))

    # Largest radius
    if "largest radius" in q or "biggest planet" in q:
        return max(candidates, key=lambda c: c["planet_profile"].get("radius_earth_radii", -1))

    # Smallest radius
    if "smallest radius" in q:
        return min(candidates, key=lambda c: c["planet_profile"].get("radius_earth_radii", 1e18))

    # fallback = semantic reasoning
    scores = [
        embed_model.encode([question + " " + c["planet_name"]])[0].sum()
        for c in candidates
    ]
    return candidates[int(np.argmax(scores))]


In [ ]:
def answer_query(question, index, meta, top_k=5):
    print("\n🔍 Query:", question)
    cands = retrieve_objects(question, index, meta, top_k)
    print("\nRetrieved:", [c["planet_name"] for c in cands])

    for p in cands:
      print(p["planet_name"], "-->", p.get("discovery", {}).get("year"))


    if not cands:
        return "No planets found."

    # ---- Year filtering ----
    year = extract_year(question.lower())
    if year:
        filtered = [c for c in cands
                    if c.get("discovery", {}).get("year") == year]

        if filtered:
            cands = filtered
            print(f"\n Filtered by discovery year = {year}")
            print("Remaining:", [c["planet_name"] for c in cands])
        else:
            print(f"\n No exact matches for discovery year {year}.")
            print("Falling back to semantic candidates.")

    # ---- Fact-check selection ----
    selected = fact_check(question, cands)

    return {
        "answer": f"The planet most aligned with your query is **{selected['planet_name']}**.",
        "planet": selected
    }


In [ ]:
# ---------- LOAD ----------
def load_planets(path):
    with open(path, "r") as f:
        return json.load(f)


# ---------- RUN ----------
planets = load_planets("/content/exoplanet_data/exoplanets.json")
index, meta = build_index(planets)


In [ ]:
out = answer_query("Which exoplanets were discovered in 2015?", index, meta)

print("\n=== FINAL OUTPUT ===")
print(out)